# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [35]:
#сумма
foldl(lambda x, y: x + y, 2, [1, 2, 3])

8

In [109]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), 'asd', [1, 2, 3])

'asd123'

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

$foldl2$

In [105]:
def foldl2(f, x0, lst):
    return foldr(lambda val, acc: lambda n: acc(f(n, val)), lambda x: x, lst)(x0)

foldl2(lambda x, y: x + y, 2, [1, 2, 3]),\
\
foldl2(lambda x, y: '{}{}'.format(x, y), 'asd', [1, 2, 3]),\
\
foldl2(lambda x, y: x / y, 1, [1, 2, 3])

(8, 'asd123', 0.16666666666666666)

$foldr2$

In [106]:
def foldr2(f, x0, lst):
    return foldl(lambda acc, val: lambda n: acc(f(val, n)), lambda x: x, lst)(x0)

foldr2(lambda x, y: x + y, 2, [1, 2, 3]),\
\
foldr2(lambda x, y: '{}{}'.format(x, y), 'asd', [1, 2, 3]),\
\
foldr2(lambda x, y: x / y, 1, [1, 2, 3])

(8, '123asd', 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

Сложность функции занимет $O(len_a*len_b)$, т.к. цикл проходится за $O(len_a)$ итераций, а сравнение списков занимает $O(len_b)$

In [102]:
import string

def check_inv(a, b):
    b_chars = dict.fromkeys(string.ascii_letters, 0)
    for char in b:
        b_chars[char] += 1
    
    
    a_chars = dict.fromkeys(string.ascii_letters, 0)
    for char in a[:len(b) - 1]:
        a_chars[char] += 1
    
    
    for i in range(len(b) - 1, len(a)):
        a_chars[a[i]] += 1
        if(a_chars == b_chars):
            return True
        a_chars[a[i - len(b) + 1]] -= 1
        
    return False

check_inv('abcrotm', 'tro'), \
\
check_inv('trdvno', 'tro'), \
\
check_inv('tlbjrlbo', 'tro')

(True, False, False)

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [86]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if(self.left == None and self.right == None):
            yield self.value
            
        if(self.left):
            yield from self.left
        if(self.right):
            yield from self.right
            
            
    
    def __str__(self):
        res = [str(self.value)]
        
        if(self.left != None):
            res.append(self.left.__str__())
        if(self.right != None):
            res.append(self.right.__str__())
            
        return ' '.join(res)
    
    
    
    def __repr__(self):
        if(self.left == None and self.right == None):
            return f"Tree({self.value})"
        
        if(self.right == None):
            return f"Tree({self.value}, {self.left.__repr__()})"
        if(self.left == None):
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
    
    
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

print(tree)
print('')
print(list(tree))
tree

0 1 3 4 2

[3, 4, 2]


Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [93]:
def parse(expr):
    buf = []

    for val in expr.split(" "):
        if(val[0] == "("):
            buf.append("(")
            buf.append(val[1:])
        elif(val[-1] == ")"):
            buf.append(val[:-1])
            buf.append(")")
        else:
            buf.append(val)

    result = []
    stack = []
    
    high_pr = set(["*", "/"])
    low_pr = set(["+", "-"])
    
    for val in buf:
        if(val.isnumeric()):
            result.append(val)
        if(val == "("):
            stack.append(val)
        if(val == ")"):
            while(stack[-1] != "("):
                result.append(stack.pop())
            stack.pop()
        if(val in low_pr):
            while(len(stack) > 0 and stack[-1] in high_pr):
                result.append(stack.pop())
            stack.append(val)
        if(val in high_pr):
            stack.append(val)
            
    while(len(stack) > 0):
        result.append(stack.pop())
    
    return result

def calc(expr):
    oper = {
        "+": lambda x, y: x + y,
        "-": lambda x, y: x - y,
        "*": lambda x, y: x * y,
        "/": lambda x, y: x / y
    }
    
    expr = parse(expr)
    stack = []
    
    for val in expr:
        if(val.isnumeric()):
            stack.append(int(val))
        if(val in oper):
            right = stack.pop()
            left = stack.pop()
            stack.append(oper[val](left, right))
    return stack.pop()

calc('2 * (15 - 3 * 4) - 2') == 4

True

In [94]:
calc('( 14 - 4 * 3 ) * 5 - 2 * ( 7 - 3 )')

2